In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import time
import string
from collections import defaultdict, deque
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# input_dir=r"C:\Users\Lenovo\Desktop\stupidcode\kaggle\jigsaw\error_analyze"
# aug1_paths=os.path.join(input_dir,f"aug_1\\data_df_aug1")
aug1_paths="./cv_analyze/luke_ruddit_0.860.csv"

In [3]:
roberta_val=pd.read_csv("./cv_analyze/roberta_0.824.csv")
def generate_comments(data):
    more_toxic_text=data[["more_toxic","encode_more","more_value"]]
    less_toxic_text=data[["less_toxic","encode_less","less_value"]]
    more_toxic_text=more_toxic_text.rename(columns={"more_toxic":"text","encode_more":"encode_text","more_value":"toxic_value"})
    less_toxic_text=less_toxic_text.rename(columns={"less_toxic":"text","encode_less":"encode_text","less_value":"toxic_value"})
    
    comments=pd.concat([more_toxic_text,less_toxic_text],axis=0)
    comments=comments.drop_duplicates(subset=None,keep="first",inplace=False) 
    return comments
def encode_label(data_df):
     # 不同训练结果下的偏差，进行一些处理 主要是luke的偏差
    comments=generate_comments(roberta_val)
    comments.index=comments["text"]
    index_encode_dict=comments.to_dict()["encode_text"]
    del comments
    # 选择直接赋值 而不再根据文本匹配 因为经过翻译后的结果可能不匹配
    encode_less_luke=data_df.apply(lambda row:index_encode_dict[row["less_toxic"]],axis=1)
    encode_more_luke=data_df.apply(lambda row:index_encode_dict[row["more_toxic"]],axis=1)
    data_df["encode_less"]=encode_less_luke
    data_df["encode_more"]=encode_more_luke
    return data_df
def sort_by_roberta(data_df):
    data_df["new_index"]=data_df.merge(roberta_val[["encode_less","encode_more","worker","Unnamed: 0"]],
         on=["encode_less","encode_more","worker"],how="left")["Unnamed: 0"]
    data_df.index=data_df["new_index"]
    data_df.sort_index(inplace=True)
    return data_df
def luke_process(data_df):
    data_df=encode_label(data_df)
    data_df=data_df.rename(columns={"less_toxic_pred":"less_value","more_toxic_pred":"more_value"})
    data_df["pair_True"]=data_df["less_value"]<data_df["more_value"]

    data_df=sort_by_roberta(data_df)
    return data_df

data_df=pd.read_csv(aug1_paths)
data_df=luke_process(data_df)

In [4]:
comments_rank=pd.read_csv("./cv_analyze/bt_work.csv")
comments_rank=comments_rank[["text","score"]]

# comments_rank.index=comments_rank.text
# text2rank=comments_rank.to_dict()["score"]

# data_df["less_value"]=data_df.apply(lambda row:text2rank[row["less_toxic"]],axis=1)
# data_df["more_value"]=data_df.apply(lambda row:text2rank[row["more_toxic"]],axis=1)
# data_df["pair_True"]=data_df.less_value<data_df.more_value

def generate_comments(data):
    more_toxic_text=data[["more_toxic","encode_more","more_value"]]
    less_toxic_text=data[["less_toxic","encode_less","less_value"]]
    more_toxic_text=more_toxic_text.rename(columns={"more_toxic":"text","encode_more":"encode_text","more_value":"toxic_value"})
    less_toxic_text=less_toxic_text.rename(columns={"less_toxic":"text","encode_less":"encode_text","less_value":"toxic_value"})
    
    comments=pd.concat([more_toxic_text,less_toxic_text],axis=0)
    comments=comments.drop_duplicates(subset=None,keep="first",inplace=False) 
    return comments
comments=generate_comments(data_df)
comments_rank=comments_rank.merge(comments[["text","encode_text"]],on="text",how="left")
less_fold=comments_rank.merge(data_df[["encode_less","fold"]],left_on="encode_text",right_on="encode_less",
                              how="left").drop_duplicates("text")["fold"]
more_fold=comments_rank.merge(data_df[["encode_more","fold"]],left_on="encode_text",right_on="encode_more",
                              how="left").drop_duplicates("text")["fold"]

less_fold=less_fold.rename("less_fold").reset_index(drop=True)
more_fold=more_fold.rename("more_fold").reset_index(drop=True)
fold_df=pd.concat([less_fold,more_fold],axis=1)
comments_rank["fold"]=fold_df.apply(lambda row:row["more_fold"] if pd.isna(row["less_fold"]) else row["less_fold"],axis=1)
comments_rank["fold"]=comments_rank.fold.astype(int)

In [8]:

# data_df=pd.read_csv(aug1_paths)
data_df["label_min"]=data_df.apply(lambda row:row["encode_less"] if row["encode_less"]<row["encode_more"] else row["encode_more"],axis=1)
data_df["label_max"]=data_df.apply(lambda row:row["encode_less"] if row["encode_less"]>row["encode_more"] else row["encode_more"],axis=1)
data_df["label_min_text"]=data_df.apply(lambda row:row["less_toxic"] if row["encode_less"]<row["encode_more"] else row["more_toxic"],axis=1)
data_df["label_max_text"]=data_df.apply(lambda row:row["less_toxic"] if row["encode_less"]>row["encode_more"] else row["more_toxic"],axis=1)
data_df["label_min_value"]=data_df.apply(lambda row:row["less_value"] if row["encode_less"]<row["encode_more"] else row["more_value"],axis=1)
data_df["label_max_value"]=data_df.apply(lambda row:row["less_value"] if row["encode_less"]>row["encode_more"] else row["more_value"],axis=1)

data_df["win_min"]=data_df.apply(lambda row:1 if row["label_min"]==row["encode_more"] else 0,axis=1)
data_df["win_max"]=data_df.apply(lambda row:0 if row["label_min"]==row["encode_more"] else 1,axis=1)

## pair True 已经失效

# data_df["pair_True"].replace({True:1,False:-1},inplace=True)

In [23]:
origin_data_df=data_df[data_df["worker"]!=999]
aug_data_df=data_df[data_df["worker"]==999]

print("origin_cv:",origin_data_df["pair_True"].mean())
print("aug_data_cv:",aug_data_df["pair_True"].mean())



origin_cv: 0.6965258403082237
aug_data_cv: nan


In [24]:
origin_data_df_agg=origin_data_df.groupby(["label_min","label_max","label_min_value","label_max_value","label_min_text","label_max_text","fold"
                                          ]).agg({"win_min":"sum","win_max":"sum"}).reset_index()
origin_data_df_agg["pair_True"]=((origin_data_df_agg.label_min_value-origin_data_df_agg.label_max_value)*(
    origin_data_df_agg.win_min-origin_data_df_agg.win_max))>0

origin_data_df_agg.pair_True.mean()

0.7434705184012663

In [25]:
# origin_error_df=origin_data_df[origin_data_df["pair_True"]==False]
# aug_error_df=aug_data_df[aug_data_df["pair_True"]==False]
# #找到所有的false 组合的情况（包括被判断正确的组合）
origin_error_df=origin_data_df_agg[origin_data_df_agg["pair_True"]==False]

## error
origin_error_df["encode_less"]=origin_error_df.apply(lambda row:row["label_min"] 
                                             if row["win_min"]<row["win_max"] else row["label_max"],axis=1)
origin_error_df["encode_more"]=origin_error_df.apply(lambda row:row["label_min"] 
                                             if row["win_min"]>row["win_max"] else row["label_max"],axis=1)

# # 76%的错误都来自 毒性相似句子 所有句子对中 52%是毒性相似句子
confuse_error_df=origin_error_df[(origin_error_df["win_min"]!=0) & (origin_error_df["win_max"]!=0)]
len(confuse_error_df)/len(origin_error_df)

C:\Users\sleep\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\sleep\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


0.7466255302738141

In [126]:
def sample_error(fold_num,origin_error_df):
    train=pd.DataFrame(columns={"text","score","encode_text","fold","sample"})
    for fold in range(fold_num):
        error_df_fold=origin_error_df[origin_error_df.fold==fold]
        comments_rank_fold=comments_rank[comments_rank.fold==fold]
        comments_rank_fold=comments_rank_fold.sort_values("score").reset_index(drop=True)
        temp_comments=comments_rank_fold.copy()
        temp_comments.index=temp_comments.encode_text
        encode2score=temp_comments.to_dict()["score"]
        del temp_comments
        temp_comments=comments_rank_fold.copy()
        temp_comments["old_index"]=temp_comments.index
        temp_comments.index=temp_comments.score
        score2index=temp_comments.to_dict()["old_index"]
        del temp_comments
        encode1=error_df_fold.encode_less.values
        encode2=error_df_fold.encode_more.values
        comments_rank_fold["sample"]=0    
        for x,y in zip(encode1,encode2):
            r1,r2=encode2score[x],encode2score[y]
            index1,index2=score2index[r1],score2index[r2]
            if index1>index2:
                index1,index2=index2,index1
            comments_rank_fold.loc[index1:index2,"sample"]+=1
        median_sample_num=comments_rank_fold["sample"].median()
        sample_comments=comments_rank_fold.query("sample>@median_sample_num")
        
        train=pd.concat([train,sample_comments])
    train=train.reset_index(drop=True)
    return train

In [127]:
train=sample_error(5,origin_error_df)
train.to_csv("./persedo_work/error_persedo.csv")

In [19]:
# fig,axs=plt.subplots(3,3,figsize=(20,10))
# sns.kdeplot(data=origin_error_df["less_value"],ax=axs[0][0])
# sns.kdeplot(data=origin_error_df["more_value"],ax=axs[1][0])
# sns.kdeplot(data=origin_error_df["more_value"]-origin_error_df["less_value"],ax=axs[2][0])

# sns.kdeplot(data=origin_data_df["less_value"],ax=axs[0][1])
# sns.kdeplot(data=origin_data_df["more_value"],ax=axs[1][1])
# sns.kdeplot(data=origin_data_df["more_value"]-origin_data_df["less_value"],ax=axs[2][1])

# sns.kdeplot(data=origin_true_df["less_value"],ax=axs[0][2])
# sns.kdeplot(data=origin_true_df["more_value"],ax=axs[1][2])
# sns.kdeplot(data=origin_true_df["more_value"]-origin_true_df["less_value"],ax=axs[2][2])

In [13]:
origin_data_df_agg=origin_data_df.groupby(["label_min","label_max","label_min_text","label_max_text"
                                        ]).agg({"win_min":"sum","win_max":"sum","label_min_value":"mean","label_max_value":"mean","pair_True":"mean"}).reset_index()
origin_data_df_agg["value_gap"]=origin_data_df_agg.apply(lambda row:row["label_max_value"]-row["label_min_value"] 
                                                         if row["win_max"]>row["win_min"] else row["label_min_value"]-row["label_max_value"],axis=1)
origin_data_df_agg["win_gap"]=origin_data_df_agg.apply(lambda row:(row["win_max"]-row["win_min"])/(row["win_max"]+row["win_min"])
                                                         if row["win_max"]>row["win_min"] else (row["win_min"]-row["win_max"])/(row["win_max"]+row["win_min"]),axis=1)

SpecificationError: nested renamer is not supported

In [149]:
#2291 条数据错误 7817条正确
origin_data_error=origin_data_df_agg[origin_data_df_agg["pair_True"]<0.5]
origin_data_true=origin_data_df_agg[origin_data_df_agg["pair_True"]>0.5]
len(origin_data_true)/len(origin_data_df_agg)

0.8396319746735259

In [150]:
# origin_data_error.to_csv(os.path.join(input_dir,f"aug_1\\origin_data_error.csv"))
# origin_data_true.to_csv(os.path.join(input_dir,f"aug_1\\origin_data_true.csv"))

In [206]:
dict(origin_data_true["win_gap"].value_counts())[1/3]

3905

In [275]:
#对正确的部分 划分为正1 正2 根据赢多少的比例来
win_gap_num=dict(origin_data_true["win_gap"].value_counts())
true_point=win_gap_num[1/3]/(win_gap_num[1/3]+win_gap_num[1])
true_val_gap=list(origin_data_true["value_gap"].sort_values())
true_point_value=true_val_gap[int(true_point*len(true_val_gap))]
origin_data_true["degree"]=origin_data_true.apply(lambda row:2 if row["value_gap"]>true_point_value else 1,axis=1)

origin_true_degree2=origin_data_true[origin_data_true["degree"]==2]
origin_true_degree1=origin_data_true[origin_data_true["degree"]==1]

D:\anaconda\envs\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [271]:
#对错误的部分 也划分为 负1 和负2
false_gap_num=dict(origin_data_error["win_gap"].value_counts())
false_point=false_gap_num[1/3]/(false_gap_num[1/3]+false_gap_num[1])
false_val_gap=list(origin_data_error["value_gap"].sort_values())
false_point_value=false_val_gap[int(false_point*len(false_val_gap))]
origin_data_error["degree"]=origin_data_error.apply(lambda row:1 if row["value_gap"]>false_point_value else 2,axis=1)

origin_false_degree2=origin_data_error[origin_data_error["degree"]==2]
origin_false_degree1=origin_data_error[origin_data_error["degree"]==1]

D:\anaconda\envs\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# 绘图